In [1]:
from dotenv import load_dotenv
from utils.agent_visualizer import print_activity

from claude_agent_sdk import ClaudeAgentOptions, ClaudeSDKClient, query

load_dotenv()

True

# 00 - 一行式研究智能体

前言：如果您还没有读过，我们强烈推荐阅读[构建有效的智能体](https://www.anthropic.com/engineering/building-effective-agents)或[我们如何构建多智能体研究系统](https://www.anthropic.com/engineering/built-multi-agent-research-system)。这些是很好的阅读材料，我们将假设您对智能体有基本了解！

在本笔记本中，我们构建我们自己的（重新）搜索智能体，这本身就是一个很好的用例，原因如下：
- 我们系统的输入不足以产生输出，这意味着需要与外部系统（例如互联网）进行交互
- 没有我们可以使用的预定义工作流程，因为我们不清楚智能体在研究过程中会发现什么

相反，研究智能体需要灵活性来探索意外线索并根据发现的内容改变方向。在其最简单形式中，研究智能体可以是一个只需搜索互联网并为您总结的智能体。

下面，我们将用几行代码实现一个基本的研究智能体。我们为Claude提供Claude Code SDK开箱即用的一个工具：[网络搜索工具](https://docs.claude.com/en/docs/agents-and-tools/tool-use/web-search-tool)。

> 在[这里](https://docs.claude.com/en/docs/claude-code/settings#tools-available-to-claude)查看Claude Code可用的工具列表

In [2]:
messages = []
async for msg in query(
    prompt="Research the latest trends in AI agents and give me a brief summary",
    options=ClaudeAgentOptions(model="claude-sonnet-4-5", allowed_tools=["WebSearch"]),
):
    print_activity(msg)
    messages.append(msg)

🤖 Thinking...
🤖 Using: WebSearch()
✓ Tool completed
🤖 Thinking...


In [ ]:
print(
    f"\nResult:\n{messages[-1].result if hasattr(messages[-1], 'result') and messages[-1].result else messages[-2].content[0].text}"
)

就这样！我们刚刚构建了一个研究智能体，它可以去浏览网络来回答（至少在其能力范围内）您抛给它的任何问题。

请注意，在我们的查询中，我们提供了`options`参数。在这里我们定义了我们智能体的配置、能力和限制。例如，我们通过传递```allowed_tool=["WebSearch"]```为我们智能体提供搜索网络的能力。

更具体地说，`allowed_tools`是一个Claude可以在无需批准的情况下使用的工具列表。其余工具仍然可用，但Claude会请求批准使用它们。也就是说，像`Read`和其他基础只读工具这样的某些工具始终是允许的。如果您想从Claude的上下文中删除任何工具，请将其添加到`disallowed_tools`中。

现在，为了更仔细地检查我们智能体采取的行动，我们提供了```visualize_conversation```函数。

In [ ]:
from utils.agent_visualizer import visualize_conversation

visualize_conversation(messages)

### 增强我们的智能体

到目前为止，我们提出了一个非常简单的（也许是天真的）实现来说明如何开始利用SDK构建研究智能体。然而，我们可以通过各种方式来改进我们的智能体，使其生产就绪。让我们涵盖其中一些：

1. 请注意之前我们只发送了一个查询？在许多系统中，人会查看系统的输出，可能会分配后续任务。就像文本补全一样，如果我们想向智能体发送多个查询（例如，1. 分析abc，2. 根据您的分析制作xyz），我们就必须将整个分析上下文复制到第二个查询中。相反，我们可以**[使用ClaudeSDKClient](https://docs.claude.com/en/docs/claude-code/sdk/sdk-python#1-the-claudesdkclient类-推荐)**来为我们维护对话上下文。

2. 引导系统的另一个好方法是**提供系统提示**，类似于文本补全使用的系统提示。要了解如何为研究智能体编写好的系统提示，我们推荐查看[这里](https://github.com/anthropics/anthropic-cookbook/tree/main/patterns/agents/prompts)。

3. **利用`Read`工具**来实现多模态输入。该工具允许Claude分析图表、信息图表和复杂系统图。

In [5]:
messages = []
async with ClaudeSDKClient(
    options=ClaudeAgentOptions(
        model="claude-sonnet-4-5",
        cwd="research_agent",
        system_prompt="You are a research agent specialized in AI",
        allowed_tools=["WebSearch", "Read"],
    )
) as research_agent:
    await research_agent.query("Analyze the chart in research_agent/projects_claude.png")
    async for msg in research_agent.receive_response():
        print_activity(msg)
        messages.append(msg)

    await research_agent.query("Use a single websearch to investigate the insights from the chart.")
    async for msg in research_agent.receive_response():
        print_activity(msg)
        messages.append(msg)

🤖 Thinking...
🤖 Using: Read()
✓ Tool completed
🤖 Thinking...
🤖 Using: Bash()
✓ Tool completed
🤖 Thinking...
🤖 Using: Read()
✓ Tool completed
🤖 Thinking...
🤖 Thinking...
🤖 Using: WebSearch()
✓ Tool completed
🤖 Thinking...


In [ ]:
visualize_conversation(messages)

### 研究智能体离开Jupyter

最后，要能够在我们的笔记本之外使用智能体，我们必须将其放入Python脚本中。我们可以在`research_agent/agent.py`中找到我们研究智能体的轻量级实现。我们定义了三个函数：
- `print_activity()` - 实时显示智能体正在执行的操作
- `get_activity_text()` - 提取自定义处理程序的活动文本
- `send_query()` - 发送和处理的查询的主要函数，带有内置活动显示

这个智能体现在可以在任何Python脚本中使用！

首先是一个测试智能体单次查询的示例：

In [ ]:
from research_agent.agent import send_query

result = await send_query("What is the Claude Code SDK? Only do one websearch and be concise")
print(f"\nResult: {result}\n")

现在我们测试一个重用同一对话的多轮对话：

In [8]:
result1 = await send_query("What is Anthropic? Only do one websearch and be concise")
print(f"\n-----\n\nInitial research: {result1}\n")

🤖 Thinking...
🤖 Using: WebSearch()
✓ Tool completed
🤖 Thinking...

-----

Initial research: Anthropic is an AI safety and research company founded in 2021 by former OpenAI researchers, including siblings Dario and Daniela Amodei. The company develops Claude, a family of large language models (LLMs) designed to be helpful, harmless, and honest.

**Key points:**
- **Mission**: Build reliable, interpretable, and steerable AI systems with a focus on AI safety
- **Main product**: Claude AI assistant (which you're currently using!)
- **Structure**: Public benefit corporation balancing profit with humanity's long-term benefit
- **Funding**: Backed by major investments from Amazon ($8B total) and Google ($2B)
- **Focus areas**: AI safety, natural language processing, human feedback, and responsible AI development

Anthropic positions itself as a "safety-first" AI lab, emphasizing the responsible development of AI systems to serve humanity's long-term well-being.



In [ ]:
# Continue the conversation to dig deeper by setting continue_conversation=True
result2 = await send_query(
    "What are some of their products?",
    continue_conversation=True,
)
print(f"\n-----\n\nFollow-up: {result2}\n")

## 结论

我们演示了Claude Code SDK如何使您仅用几行代码就能构建一个功能研究智能体。通过利用内置的WebSearch工具，我们创建了一个能够自主信息收集和综合的智能体。我们还探索了ClaudeSDKClient如何维护多个查询之间的对话上下文，以及如何通过Read工具融入多模态能力。

基本智能体工作流程这一基础为您提供了更复杂的实现准备。在下一个笔记本中，我们将继续构建一个首席助理智能体，它协调多个专门的子智能体，为不同利益相关者实现自定义输出样式，并使用钩子进行治理和合规跟踪。

下一步：[01_The_chief_of_staff_agent.ipynb](01_The_chief_of_staff_agent.ipynb) - 学习如何编排具有企业级功能的复杂多智能体系统。